In [1]:
import numpy as np
import pandas as pd
import os
import re
import json
import copy
import re

**********************************************************************************************************************

In this file we aim to extract all disorder information from .cif files. We focus on classification of orbits with respect to 3 types of structural disorder: substitutional, vacancies and positional disorder. Correspondingly orbits can have types: O, S, V, P, SV, SP, VP, SVP. Compounds are classified according to the type of orbits they contain. 

After extraction of disorder information we calculate mixing and configurational entropy. The main difference (and novelty) in calculation of mixing and configurational entropy is recognition of the fact that positional disorder does not contribute to the mixing entropy, but contributes to configurational entropy. Due to developed orbit classification, entropy calculations from cif files are more accurate than before (for example, as implemented in crystIT package).

Additionally, our classification allows more accurate calculation of the number of disordered sites/orbits, which is another measure of disorder which we use in the work. To calculate this property we consider "effective multiplicity" of positionally disordered orbits. For example, for positionally disordered orbits of P, VP, SP, SPV type with internal intersection we can calculate the effective multiplcity as the number of connected components within the graph formed by the set of sites of this orbit, assuming that two sites are connected if they intersect. For P, VP, SP, SPV orbits with external intersection, the effective number of orbits is reduced. We assume that intersecting orbits form one delocalised orbit. In mixed cases of internal and external intersection, the picture describing orbits and their multiplicities can be very complicated. 

**********************************************************************************************************************

In [2]:
def float_brackets(s):
    s_new=''
    for sym in s:
        if(sym=='(' or sym==')' or sym=="'"):
            pass
        else:
            s_new+=sym
    return float(s_new)

In [3]:
radius=pd.read_csv('Radiuses-crystal-reduced.csv')
dict_el=list(set(radius['element'].values))

In [4]:
# implement check of the multiplicities and reading Wyckoff positions

In [5]:
from pymatgen.core.structure import Structure
from pymatgen.analysis.local_env import CrystalNN
import networkx as nx

def disorder_type(file, cutoff='2mean',tol=0.5):    
    
    # read the structure from cif file
    struct = Structure.from_file(file,occupancy_tolerance=1.05)
    # merging sites is used to recognise orbits occupied by > 1 elements as single orbits
    try:
        struct.merge_sites(tol=0.005)
    except:
        pass
    species=[]
    orbits=[]
    total_site_occ=[]
    for i in range(len(struct)):
        sp=struct.sites[i].species.as_dict()
        # names of species occupying sites
        species.append(sp)
        # list of orbits for each site
        orbits.append(struct.sites[i].label)
        # sum occupancies by different elements
        total_site_occ.append(sum(sp.values()))
    
    # Taking the set from the list of orbits. Note if different orbits have the same name, they will be merged
    # as Wyckoff symbols are not present in Structure datatype. Mulitplicities will be summed.
    set_orb=set(orbits)
    mult={}
    for orb in set_orb:
        mult[orb]=0
    for orb in orbits:
        mult[orb]+=1
    multi=[]
    for orb in orbits:
        multi.append(mult[orb])
    
    # sometimes oxidations are non-integer (signifying the presence of disorder which we do not consider)
    # but for the table look up we need integer oxidation numbers
    elements=[]
    oxidations_rounded=[]
    for sp in species:
        orb_elem=[]
        orb_oxi=[]
        for spk in sp.keys():
            elem=[]
            for el in dict_el:
                if(re.search(el, spk)):
                    elem.append(el)
            elem_final=elem[0]
            for el in elem:
                if(len(el)>len(elem_final)):
                    elem_final=el

            n=len(elem_final)
            if(len(spk[n:-1])>0):
                oxi=int(round(float(spk[n:-1]),0))
            else:
                oxi=1
            oxi=str(oxi)+spk[-1:]
            orb_elem.append(elem_final)
            orb_oxi.append(oxi)
        elements.append(orb_elem)
        oxidations_rounded.append(orb_oxi)
    
    # determining intersecting the list of intersecting sites for each site
    distance_template=np.zeros((len(struct),len(struct)))
    if(cutoff=='1max'):
        for i in range(len(struct)):
            rad=[]
            for j,el in enumerate(elements[i]):
                da=radius.loc[radius['element']==el]
                db=da.loc[da['oxi_state']==oxidations_rounded[i][j]]
                if(len(db)>0):
                    rad.append(db['max_radius'].values[0])
                else:
                    default_rad=max(da['max_radius'].values)
                    rad.append(default_rad)
            ri=np.max(rad)
            for j in range(len(struct)):
                if(i!=j):
                    distance_template[i,j]=ri

    if(cutoff=='2mean'):
        mean_rad=np.zeros(len(struct))
        for i in range(len(struct)):
            rad=[]
            for k,el in enumerate(elements[i]):
                da=radius.loc[radius['element']==el]
                db=da.loc[da['oxi_state']==oxidations_rounded[i][k]]
                if(len(db)>0):
                    rad.append(db['max_radius'].values[0])
                else:
                    default_rad=max(da['max_radius'].values)
                    rad.append(default_rad)
            mean_rad[i]=np.mean(rad)
        for i in range(len(struct)):
            ri=mean_rad[i]
            for j in range(len(struct)):
                rj=mean_rad[j]
                if(i!=j):
                    distance_template[i,j]=(ri+rj)*tol
    
    dist=struct.distance_matrix
    a=dist-distance_template
    close=[]
    for i in range(len(struct)):
        cl=[]
        for j in range(len(struct)):
            if(a[i,j]<0):
                cl.append(j)
        close.append(np.array(cl,dtype=int))
    
    # dataframe with information about sites
    sites=struct.as_dataframe()
    sites['species']=species
    sites['orbits']=orbits
    sites['total_site_occ']=total_site_occ
    sites['mult']=multi
    sites['intersection']=close
    
    set_orbits=list(set(sites['orbits']))
    
    # create a list of elements on each orbit
    el_orbits=[]
    for orb in set_orbits:
        el_orbits.append(list(sites.loc[sites['orbits']==orb]['species'].values[0].keys()))
    
    # determine intersecting orbits, internal intersection, intersect_index
    intersect_index=[]
    internal_intersection=[]
    intersect_orbit=[]
    for orb in set_orbits:
        db=sites.loc[sites['orbits']==orb]
        ib=[]
        switch=0
        for j in db['intersection'].values:
            for i in j:
                # so if there are intersection and (1) there is intersection with the site form the same orbit
                # then internal)intersection = True
                # (2) there is intersection with site from another orbit, the name of that orbit is added to 
                # the list of intersecting orbits 
                if(i not in db.index.values):
                    ib.append(sites.iloc[i]['orbits'])
                elif(i in db.index.values):
                    switch=1
        if(switch==0):
            # only external intersection
            internal_intersection.append(False)
            intersect_index.append([1])
        elif(switch!=0 and len(ib)==0):
            # only internal intersection
            internal_intersection.append(True)
            index=sites.loc[sites['orbits']==orb].index.values
            intersections=sites.loc[sites['orbits']==orb]['intersection'].values
            # building a praph of nodes
            G = nx.Graph()
            for ie in range(len(index)):
                G.add_node(index[ie])
                for je in intersections[ie]:
                    G.add_node(je)
                    G.add_edge(index[ie],je)
            inter_sizes=list(set([len(i) for i in nx.connected_components(G)]))
            intersect_index.append(inter_sizes)
        else:
            # internal and external intersection
            # in this case the list of itersecting orbits contains the orbit itself
            internal_intersection.append(True)
            intersect_index.append([np.nan])
              
        ib=list(set(ib))
        intersect_orbit.append(ib)
    
    # to refine the intersecting orbit determination we constract a graph where orbits are nodes 
    # and there is an edge between any two intersecting orbits. The connected components within this graph
    # show the list of 'effective' orbits in the compound.
    
    intersect_orbit_connected=[]
    G=nx.Graph()
    for orb in set_orbits:
        G.add_node(orb)
        for je in intersect_orbit[set_orbits.index(orb)]:
            G.add_edge(orb,je)
    connected_comp=[]
    for i in nx.connected_components(G):
        connected_comp.append(i)
    
    for orb in set_orbits:
        for jj in connected_comp:
            if(orb in jj):
                intersect_orbit_connected.append(jj)
    
    # calculating multiplicities and total_occupancies of intersect_orbit_connected
    intersect_orbit_connected_mult=[]
    intersect_orbit_connected_occ=[]

    for ind,s_orb in enumerate(intersect_orbit_connected):
        s_orb=list(s_orb)
        index=sites.loc[sites['orbits']==s_orb[0]].index.values
        intersections=sites.loc[sites['orbits']==s_orb[0]]['intersection'].values
        for ind in range(1,len(s_orb)):
            index=np.append(index,sites.loc[sites['orbits']==s_orb[ind]].index.values)
            intersections=np.append(intersections,sites.loc[sites['orbits']==s_orb[ind]]['intersection'].values)

        G = nx.Graph()
        for ie in range(len(index)):
            G.add_node(index[ie])
            for je in intersections[ie]:
                G.add_node(je)
                G.add_edge(index[ie],je)
        cc_comp_mult=[]
        cc_comp_occ=[]
        for i in nx.connected_components(G):
            cc_comp_mult.append(len(i))
            m=0
            for j in i:
                m+=sites.iloc[j]['total_site_occ']
            cc_comp_occ.append(round(m,2))
        intersect_orbit_connected_mult.append(len(cc_comp_mult))
        if(len(set(cc_comp_occ))==1):
            intersect_orbit_connected_occ.append(cc_comp_occ[0])
        else:
            intersect_orbit_connected_occ.append(np.nan)
    
    # list of disorder types of orbits
    orbit_disorder=[]
    # magic number is a sum of all occupancies of sites of the orbit
    # used to ditinguish vacancies and positional disorder
    magic_number=[]
    
    # magic number is a sum of all occupancies of sites of the orbit
    # used to ditinguish vacancies and positional disorder
    for orb in set_orbits:
        dc=sites.loc[sites['orbits']==orb]
        magic_number.append(sum(dc['total_site_occ'].values))
    
    #  building a classifier
    orb_occ=[]
    for i,orb in enumerate(set_orbits):
        db=sites.loc[sites['orbits']==orb]
        # orbits without positional disorder component
        if(db['total_site_occ'].values[0]==1 and len(db['Species'].values[0])==1):
            orbit_disorder.append('O') #1
            orb_occ.append(1.0)
        elif(db['total_site_occ'].values[0]<1 and len(db['Species'].values[0])==1 and 
             internal_intersection[i]==False and len(intersect_orbit_connected[i])==1):
            orbit_disorder.append('V') #2
            orb_occ.append(db['total_site_occ'].values[0])
        elif(db['total_site_occ'].values[0]>0.989 and len(db['Species'].values[0])>1 and 
             internal_intersection[i]==False and len(intersect_orbit_connected[i])==1):
            orbit_disorder.append('S') #3
            orb_occ.append(1.0)
        elif(db['total_site_occ'].values[0]<0.989 and len(db['Species'].values[0])>1 and 
             internal_intersection[i]==False and len(intersect_orbit_connected[i])==1):
            orbit_disorder.append('SV') #4
            orb_occ.append(db['total_site_occ'].values[0])
        
        # orbits with positional disorder componenet: have only internal intersection  
        elif(internal_intersection[i]==True and len(intersect_orbit_connected[i])==1):
            if(len(el_orbits[i])>1):
                if(abs(magic_number[i]-round(magic_number[i],0))<0.011):
                    orbit_disorder.append('SP') #7
                else:
                    orbit_disorder.append('SVP') #8
            else:
                if(abs(magic_number[i]-round(magic_number[i],0))<0.005):
                    orbit_disorder.append('P') #5
                else:
                    orbit_disorder.append('VP') #6
        
        # orbits with positional disorder componenet: have external intersection and/no internal intersection             
        elif(len(intersect_orbit_connected[i])>1):
            intersect_el=[]
            for intersect_orb in intersect_orbit_connected[i]:
                for j,orb in enumerate(set_orbits):
                    if(orb==intersect_orb):
                        for el in el_orbits[j]:
                            intersect_el.append(el)
            intersect_el=list(set(intersect_el))
    
            if(el_orbits[i]==intersect_el):
                if(len(el_orbits[i])>1):
                    if(abs(intersect_orbit_connected_occ[i]-round(intersect_orbit_connected_occ[i],0))<0.011):
                        orbit_disorder.append('SP') #11
                    elif(np.isnan(intersect_orbit_connected_occ[i])):
                        orbit_disorder.append('COM') #13
                    else:
                        orbit_disorder.append('SVP') #12                      
                else:
                    if(abs(intersect_orbit_connected_occ[i]-round(intersect_orbit_connected_occ[i],0))<0.011):
                        orbit_disorder.append('P') #9
                    else:
                        orbit_disorder.append('VP') #10
            else:
                if(abs(intersect_orbit_connected_occ[i]-round(intersect_orbit_connected_occ[i],0))<0.011):
                    orbit_disorder.append('SP') #11
                elif(np.isnan(intersect_orbit_connected_occ[i])):
                        orbit_disorder.append('COM') #13
                else:
                    orbit_disorder.append('SVP') #12

    return intersect_orbit_connected,intersect_index,orbit_disorder,set_orbits, mult, intersect_orbit,internal_intersection, magic_number,intersect_orbit_connected_mult,intersect_orbit_connected_occ

In [6]:
path='...' # path should lead to a folder with CIFs
list_of_files=os.listdir(path)

In [7]:
# path='/Users/elenapatyukova/Documents/Disorder-prediction/datasets/Examples/'
# list_of_files=os.listdir(path)

In [8]:
# try to read with ASE and reexport in pymatgen
# struct=Structure.from_file(path+list_of_files[1])

In [57]:
disordered_comp=[]
difficult_cifs=[]
other=[]
exc=[]

for file in list_of_files[180000:]:
    try:
        with open(path+file) as f:
            switch=0
            switch1=0
            formula_next_line=0
            compound={}
            elements={}
            oxidations={}

            for line in f:
                if(formula_next_line==1):
                    compound['Formula']=line[1:-2]
                    formula_next_line=0
                    if(len(compound['Formula'])==0):
                        difficult_cifs.append(file)
                if(line[:21]=="_chemical_formula_sum"):
                    compound['Formula']=line[22:-1]
                    if(len(compound['Formula'])>3):
                        compound['Formula']=line[23:-2]
                    if(len(compound['Formula'])==0):
                        formula_next_line=1
                if(line[:19]=="_database_code_ICSD"):
                    compound['ID']=line[20:-1]
                if(line[:25]=='_space_group_name_H-M_alt'):
                    compound['symmetry']=line[27:-2]
                if(line[:22]=='_space_group_IT_number'):
                    compound['group_num']=int(line[23:-1])
                if(line[:21]=='_cell_formula_units_Z'):
                    compound['Z']=int(line[22:-1])
            
            intersect_orbit_connected,intersect_index,orbit_disorder,set_orbits, multi, intersect_orbit,internal_intersection,magic_number, intersect_orbit_connected_mult,intersect_orbit_connected_occ=disorder_type(path+file,cutoff='2mean',tol=0.5)
            compound['disorder_type']=orbit_disorder
            compound['multiplicity']=multi
            compound['orbits']=set_orbits
            compound['intersect_orbit']=intersect_orbit
            compound['internal_intersection']=internal_intersection
            compound['magic_number']=magic_number
            compound['intersect_orbit_connected']=intersect_orbit_connected
            compound['intersect_index']=intersect_index
            compound['intersect_orbit_connected_mult']=intersect_orbit_connected_mult
            compound['intersect_orbit_connected_occ']=intersect_orbit_connected_occ
            
            if(len(compound['Formula'])>1 and len(orbit_disorder)>0):                
                disordered_comp.append(compound)
             
            else:
                if(file not in difficult_cifs):
                    other.append(file)
                              
    except:
#         print(file)
#         print(x,x[8])
        exc.append(file)

/Users/elenapatyukova/anaconda3/envs/py3/lib/python3.11/site-packages/pymatgen/io/cif.py:993: UserWarning: Some occupancies ([1.003, 0.999, 1.0]) sum to > 1! If they are within the occupancy_tolerance, they will be rescaled. The current occupancy_tolerance is set to: 1.05
  warnings.warn(msg)
/Users/elenapatyukova/anaconda3/envs/py3/lib/python3.11/site-packages/pymatgen/io/cif.py:1145: UserWarning: Issues encountered while parsing CIF: Some occupancies ([1.003, 0.999, 1.0]) sum to > 1! If they are within the occupancy_tolerance, they will be rescaled. The current occupancy_tolerance is set to: 1.05
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))
/Users/elenapatyukova/anaconda3/envs/py3/lib/python3.11/site-packages/pymatgen/io/cif.py:993: UserWarning: Some occupancies ([0.998, 1.001, 1.034]) sum to > 1! If they are within the occupancy_tolerance, they will be rescaled. The current occupancy_tolerance is set to: 1.05
  warnings.warn(msg)
/Users/elenap

/Users/elenapatyukova/anaconda3/envs/py3/lib/python3.11/site-packages/pymatgen/io/cif.py:1145: UserWarning: Issues encountered while parsing CIF: Some fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))
/Users/elenapatyukova/anaconda3/envs/py3/lib/python3.11/site-packages/pymatgen/io/cif.py:1145: UserWarning: Issues encountered while parsing CIF: Some fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))
/Users/elenapatyukova/anaconda3/envs/py3/lib/python3.11/site-packages/pymatgen/io/cif.py:1145: UserWarning: Issues encountered while parsing CIF: Some fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))
/Users/elenapatyukova/anaconda3/en

/Users/elenapatyukova/anaconda3/envs/py3/lib/python3.11/site-packages/pymatgen/io/cif.py:1145: UserWarning: Issues encountered while parsing CIF: Some fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))
/Users/elenapatyukova/anaconda3/envs/py3/lib/python3.11/site-packages/pymatgen/io/cif.py:1145: UserWarning: Issues encountered while parsing CIF: Some fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))
/Users/elenapatyukova/anaconda3/envs/py3/lib/python3.11/site-packages/pymatgen/io/cif.py:993: UserWarning: Some occupancies ([0.562, 1.067, 0.97, 1.004, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]) sum to > 1! If they are within the occupancy_tolerance, they will be rescaled. The current occupancy_tolerance is set to: 1.05
  warnings.warn(msg)
/Users/elenapa

/Users/elenapatyukova/anaconda3/envs/py3/lib/python3.11/site-packages/pymatgen/io/cif.py:1145: UserWarning: Issues encountered while parsing CIF: Some fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))
/Users/elenapatyukova/anaconda3/envs/py3/lib/python3.11/site-packages/pymatgen/io/cif.py:1145: UserWarning: Issues encountered while parsing CIF: Some fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))
/Users/elenapatyukova/anaconda3/envs/py3/lib/python3.11/site-packages/pymatgen/io/cif.py:1145: UserWarning: Issues encountered while parsing CIF: Some fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))
/Users/elenapatyukova/anaconda3/en

/Users/elenapatyukova/anaconda3/envs/py3/lib/python3.11/site-packages/pymatgen/io/cif.py:1145: UserWarning: Issues encountered while parsing CIF: Some fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))
/Users/elenapatyukova/anaconda3/envs/py3/lib/python3.11/site-packages/pymatgen/io/cif.py:1145: UserWarning: Issues encountered while parsing CIF: Some fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))
/Users/elenapatyukova/anaconda3/envs/py3/lib/python3.11/site-packages/pymatgen/io/cif.py:1145: UserWarning: Issues encountered while parsing CIF: Some fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))
/Users/elenapatyukova/anaconda3/en

...

/Users/elenapatyukova/anaconda3/envs/py3/lib/python3.11/site-packages/pymatgen/io/cif.py:1145: UserWarning: Issues encountered while parsing CIF: Some fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))
/Users/elenapatyukova/anaconda3/envs/py3/lib/python3.11/site-packages/pymatgen/io/cif.py:1145: UserWarning: Issues encountered while parsing CIF: Some fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))
/Users/elenapatyukova/anaconda3/envs/py3/lib/python3.11/site-packages/pymatgen/io/cif.py:1145: UserWarning: Issues encountered while parsing CIF: Some fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))
/Users/elenapatyukova/anaconda3/en

/Users/elenapatyukova/anaconda3/envs/py3/lib/python3.11/site-packages/pymatgen/io/cif.py:1145: UserWarning: Issues encountered while parsing CIF: Some fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))
/Users/elenapatyukova/anaconda3/envs/py3/lib/python3.11/site-packages/pymatgen/io/cif.py:1145: UserWarning: Issues encountered while parsing CIF: Some fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))
/Users/elenapatyukova/anaconda3/envs/py3/lib/python3.11/site-packages/pymatgen/io/cif.py:1145: UserWarning: Issues encountered while parsing CIF: Some fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))
/Users/elenapatyukova/anaconda3/en

/Users/elenapatyukova/anaconda3/envs/py3/lib/python3.11/site-packages/pymatgen/io/cif.py:1145: UserWarning: Issues encountered while parsing CIF: Some fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))
/Users/elenapatyukova/anaconda3/envs/py3/lib/python3.11/site-packages/pymatgen/io/cif.py:213: UserWarning: Possible issue in CIF file at line: Sn
  warnings.warn(f"Possible issue in CIF file at line: {issue}")
/Users/elenapatyukova/anaconda3/envs/py3/lib/python3.11/site-packages/pymatgen/io/cif.py:213: UserWarning: Possible issue in CIF file at line: phase'
  warnings.warn(f"Possible issue in CIF file at line: {issue}")
/Users/elenapatyukova/anaconda3/envs/py3/lib/python3.11/site-packages/pymatgen/io/cif.py:1145: UserWarning: Issues encountered while parsing CIF: Some fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("

/Users/elenapatyukova/anaconda3/envs/py3/lib/python3.11/site-packages/pymatgen/io/cif.py:1145: UserWarning: Issues encountered while parsing CIF: Some fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))
/Users/elenapatyukova/anaconda3/envs/py3/lib/python3.11/site-packages/pymatgen/io/cif.py:1145: UserWarning: Issues encountered while parsing CIF: Some fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))
/Users/elenapatyukova/anaconda3/envs/py3/lib/python3.11/site-packages/pymatgen/io/cif.py:1145: UserWarning: Issues encountered while parsing CIF: Some fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))
/Users/elenapatyukova/anaconda3/en

/Users/elenapatyukova/anaconda3/envs/py3/lib/python3.11/site-packages/pymatgen/io/cif.py:1145: UserWarning: Issues encountered while parsing CIF: Some fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))
/Users/elenapatyukova/anaconda3/envs/py3/lib/python3.11/site-packages/pymatgen/io/cif.py:1145: UserWarning: Issues encountered while parsing CIF: Some fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))
/Users/elenapatyukova/anaconda3/envs/py3/lib/python3.11/site-packages/pymatgen/io/cif.py:1145: UserWarning: Issues encountered while parsing CIF: Some fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))


In [58]:
for file in difficult_cifs:
    try:
        with open(path+file) as f:
            switch=0
            swithch1=0
            formula_switch=0
            compound={}
            elements={}
            oxidations={}
            formula=''
            
            info=[]
            for line in f:
                if(formula_switch==1 and line[:14]!="_chemical_name" and line[:1]!=';'):
                    formula=formula+' '+line[:-1]
                if(line[:14]=="_chemical_name"):
                    formula_switch=0
                if(line[:21]=="_chemical_formula_sum"):
                    formula_switch=1
                if(line[:25]=='_space_group_name_H-M_alt'):
                    compound['symmetry']=line[27:-2]
                if(line[:22]=='_space_group_IT_number'):
                    compound['group_num']=int(line[23:-1])
                if(line[:19]=="_database_code_ICSD"):
                    compound['ID']=line[20:-1]
                if(line[:21]=='_cell_formula_units_Z'):
                    compound['Z']=int(line[22:-1])
                if(line[:5]=='loop_' or line[:14]=='#End of TTdata'):
                    switch=0
                    switch1=0
                if(switch==1):
                    x=line.split()
                    for sym in x:
                        if(sym=="'"):
                            x.remove("'")
                    key=x[1][:-2]
                    if key in elements.keys():
                        elements[key].append((float_brackets(x[-1]),int(x[2])))
                    else:
                        elements[key]=[(float_brackets(x[-1]),int(x[2]))]
                    param=[key,int(x[2]),x[3],float_brackets(x[4]),
                           float_brackets(x[5]),float_brackets(x[6]),
                           float_brackets(x[-1])]
                    info.append(param)
                if(switch1==1):
                    x=line.split()
                    for sym in x:
                        if(sym=="'"):
                            x.remove("'")
                    key=x[0][:-2]
                    if key in oxidations.keys():
                        oxidations[key].append(float_brackets(x[-1]))
                    else:
                        oxidations[key]=[float_brackets(x[-1])]

                if(line[:20]=='_atom_site_occupancy'):
                    switch=1   
                if(line[:27]=='_atom_type_oxidation_number'):
                    switch1=1
                
            compound['Formula']=formula[1:]                                      
                
            intersect_orbit_connected,intersect_index,orbit_disorder,set_orbits, mult, intersect_orbit,internal_intersection,vacancy_number=disorder_type(path+file,cutoff='2mean',tol=0.5)
            compound['disorder_type']=orbit_disorder
            compound['multiplicity']=multi
            compound['orbits']=set_orbits
            compound['intersect_orbit']=intersect_orbit
            compound['internal_intersection']=internal_intersection
            compound['magic_number']=magic_number
            compound['intersect_orbit_connected']=intersect_orbit_connected
            compound['intersect_index']=intersect_index
            compound['intersect_orbit_connected_mult']=intersect_orbit_connected_mult
            compound['intersect_orbit_connected_occ']=intersect_orbit_connected_occ

            disordered_comp.append(compound)

            
    except:
#         print(file)
#         print(x,x[8])
        exc.append(file)

/Users/elenapatyukova/anaconda3/envs/py3/lib/python3.11/site-packages/pymatgen/io/cif.py:993: UserWarning: Some occupancies ([1.0, 1.0001, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]) sum to > 1! If they are within the occupancy_tolerance, they will be rescaled. The current occupancy_tolerance is set to: 1.05
  warnings.warn(msg)
/Users/elenapatyukova/anaconda3/envs/py3/lib/python3.11/site-packages/pymatgen/io/cif.py:1145: UserWarning: Issues encountered while parsing CIF: Some occupancies ([1.0, 1.0001, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]) sum to > 1! If they are within the occupancy_tolerance, they will be rescaled. The current occupancy_tolerance is set to: 1.05
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))
/Users/elenapatyukova/anaconda3/envs/py3/lib/python3.11/site-packages/pymatgen/io/cif.py:993: UserWarning: Some occupancies ([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.028, 1.0, 1.0, 1.0050000000000001, 0.08]) sum to > 1! If they a

/Users/elenapatyukova/anaconda3/envs/py3/lib/python3.11/site-packages/pymatgen/io/cif.py:1145: UserWarning: Issues encountered while parsing CIF: Some fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))
/Users/elenapatyukova/anaconda3/envs/py3/lib/python3.11/site-packages/pymatgen/io/cif.py:993: UserWarning: Some occupancies ([0.904, 1.0010000000000001, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]) sum to > 1! If they are within the occupancy_tolerance, they will be rescaled. The current occupancy_tolerance is set to: 1.05
  warnings.warn(msg)
/Users/elenapatyukova/anaconda3/envs/py3/lib/python3.11/site-packages/pymatgen/io/cif.py:1145: UserWarning: Issues encountered while parsing CIF: Some occupancies ([0.904, 1.0010000000000001, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]) sum to > 1! If they are within the occupancy_tolerance, they wi

/Users/elenapatyukova/anaconda3/envs/py3/lib/python3.11/site-packages/pymatgen/io/cif.py:993: UserWarning: Some occupancies ([0.9994999999999999, 1.0, 0.5, 1.0139999999999998, 1.0, 1.0, 1.0, 1.0]) sum to > 1! If they are within the occupancy_tolerance, they will be rescaled. The current occupancy_tolerance is set to: 1.05
  warnings.warn(msg)
/Users/elenapatyukova/anaconda3/envs/py3/lib/python3.11/site-packages/pymatgen/io/cif.py:1145: UserWarning: Issues encountered while parsing CIF: Some occupancies ([0.9994999999999999, 1.0, 0.5, 1.0139999999999998, 1.0, 1.0, 1.0, 1.0]) sum to > 1! If they are within the occupancy_tolerance, they will be rescaled. The current occupancy_tolerance is set to: 1.05
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))
/Users/elenapatyukova/anaconda3/envs/py3/lib/python3.11/site-packages/pymatgen/io/cif.py:993: UserWarning: Some occupancies ([0.97, 1.0, 1.001, 1.0, 1.0, 0.999, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]) sum t

/Users/elenapatyukova/anaconda3/envs/py3/lib/python3.11/site-packages/pymatgen/io/cif.py:1145: UserWarning: Issues encountered while parsing CIF: Some fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))
/Users/elenapatyukova/anaconda3/envs/py3/lib/python3.11/site-packages/pymatgen/io/cif.py:1145: UserWarning: Issues encountered while parsing CIF: Some fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))
/Users/elenapatyukova/anaconda3/envs/py3/lib/python3.11/site-packages/pymatgen/io/cif.py:1145: UserWarning: Issues encountered while parsing CIF: Some fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))
/Users/elenapatyukova/anaconda3/en

In [59]:
print(len(disordered_comp))
print(len(other))
print(len(exc))
print(len(difficult_cifs))

31724
0
710
79


Exploaration of the files in exc list shows that there are two main reasons: <br>
(1) sum of occupancies exceeds the tollerance <br>
(2) structure contains D

In [60]:
ids=[]
formulas=[]
dis=[]
set_dis=[]
sym=[]
num=[]
sites=[]
mixing=[]
element_dis=[]
z=[]
elements_on_orbits=[]
num_dis_sites=[]
num_sites=[]
vacancy_parameter=[]
orbits=[]
multi=[]
intersect_index=[]

intersect_orbit=[]
internal_inter=[]
magic_number=[]
intersect_orbit_connected=[]

intersect_orbit_connected_mult=[]
intersect_orbit_connected_occ=[]

for i in disordered_comp:
    ids.append(int(i['ID']))
    formulas.append(i['Formula'])
    dis.append(i['disorder_type'])
    set_dis.append(set(i['disorder_type']))
    sym.append(i['symmetry'])
    num.append(i['group_num'])
    z.append(i['Z'])
    orbits.append(i['orbits'])
    multi.append(i['multiplicity'])
    
    intersect_orbit.append(i['intersect_orbit'])
    internal_inter.append(i['internal_intersection'])
    magic_number.append(i['magic_number'])
    intersect_orbit_connected.append(i['intersect_orbit_connected'])
    intersect_index.append(i['intersect_index'])
    intersect_orbit_connected_mult.append(i['intersect_orbit_connected_mult'])
    intersect_orbit_connected_occ.append(i['intersect_orbit_connected_occ'])

In [61]:
df=pd.DataFrame()
df['ID']=ids
df['Formula']=formulas
df['disorder_type']=dis
df['disorder_set']=set_dis
df['symmetry']=sym
df['group_num']=num
df['Z']=z
df['orbits']=orbits
df['multiplicity']=multi
df['intersect_orbit']=intersect_orbit
df['internal_intersection']=internal_inter
df['magic_number']=magic_number
df['intersect_orbit_connected']=intersect_orbit_connected
df['intersect_index']=intersect_index
df['intersect_orbit_connected_mult']=intersect_orbit_connected_mult
df['intersect_orbit_connected_occ']=intersect_orbit_connected_occ

In [62]:
from pymatgen.core.composition import Composition

mixing_entropy=0
conf_entropy=0

calculated_orbits=np.zeros(len(orbits))

def get_occupancies(orb):
    species=re.split(',',orb)
    names=[]
    occ=[]
    for sp in species:
        parts=re.split(':',sp)
        names.append(parts[0])
        occ.append(float(parts[1]))
    return names,occ

def entropy_calulation(df):
    mixing_entropy=np.zeros(len(df))
    conf_entropy=np.zeros(len(df))
    
    for n in range(len(df)):
        try:
            mult=df.iloc[n]['multiplicity']
            orbits=df.iloc[n]['orbits']
            dis_type=df.iloc[n]['disorder_type']
            intersect_orbit=df.iloc[n]['intersect_orbit']
            intersect_orbit_connected=df.iloc[n]['intersect_orbit_connected']
            internal_intersection=df.iloc[n]['internal_intersection']
            intersect_index=df.iloc[n]['intersect_index']
            intersect_orbit_connected_occ=df.iloc[n]['intersect_orbit_connected_occ']
            intersect_orbit_connected_mult=df.iloc[n]['intersect_orbit_connected_mult']
            z=df.iloc[n]['Z']

            calculated_orbits=np.zeros(len(orbits))

            for i,orb in enumerate(orbits):
                # standard orbits with substitutional disorder and vacancies
                
                if(dis_type[i]=='O'):
                    calculated_orbits[i]=1
                elif(dis_type[i]=='S'):
                    _,occ=get_occupancies(orb)
                    for j in occ:
                        mixing_entropy[n]+=-mult[orb]*j*np.log(j)
                        conf_entropy[n]+=-mult[orb]*j*np.log(j)
                    calculated_orbits[i]=1
                elif(dis_type[i]=='V'):
                    _,occ=get_occupancies(orb)
                    sum_occ=np.sum(occ)
                    occ.append(1-sum_occ)
                    for j in occ:
                        mixing_entropy[n]+=-mult[orb]*j*np.log(j)
                        conf_entropy[n]+=-mult[orb]*j*np.log(j)
                    calculated_orbits[i]=1
                elif(dis_type[i]=='SV'):
                    _,occ=get_occupancies(orb)
                    sum_occ=np.sum(occ)
                    occ.append(1-sum_occ)
                    for j in occ:
                        mixing_entropy[n]+=-mult[orb]*j*np.log(j)
                        conf_entropy[n]+=-mult[orb]*j*np.log(j)
                    calculated_orbits[i]=1
                elif(dis_type[i]=='COM'):
                    mixing_entropy[n]=np.nan
                    conf_entropy[n]=np.nan


                # for orbits with positional disorder expressions for mixing and configurational entropy will be different    
                elif(dis_type[i]=='SP'):
                    # as intersecting orbits appear >1 time in the list of orbits
                    if(calculated_orbits[i]==0):
                        if(internal_intersection[i]==True and len(intersect_orbit_connected[i])==1):
                            # internal intersection, no external intersection
                            calculated_orbits[i]=1
                            _,occ=get_occupancies(orb)
                            for j in occ:
                                mixing_entropy[n]+=-mult[orb]*j*np.log(j*intersect_index[i][0])
                                conf_entropy[n]+=-mult[orb]*j*np.log(j)
                        
                        elif(len(intersect_orbit_connected[i])>1 and intersect_orbit_connected_occ[i]<=1.02):
                            # no internal intersection, so multiplicities as they are in mult
                            # but we may have external intersection of >1 S orbits
                            mixing_occ={}
                            for orb1 in intersect_orbit_connected[i]:
                                k=orbits.index(orb1)
                                if(internal_intersection[k]==False):
                                    if(calculated_orbits[k]==0):
                                        calculated_orbits[k]=1
                                        names,occ=get_occupancies(orb1)
                                        for j in occ:
                                            conf_entropy[n]+=-intersect_orbit_connected_mult[k]*j*np.log(j)
                                # we need mixing_occ because different intersecting orbits may have the same element
                                # and for calculation of mixing entropy we need to sum all occupancies for it on different orbits
                                        for ind,name in enumerate(names):
                                            if(name in mixing_occ.keys()):
                                                mixing_occ[name]+=occ[ind]
                                            else:
                                                mixing_occ[name]=occ[ind]
                                    
                                else:
                                    if(calculated_orbits[k]==0):
                                        calculated_orbits[k]=1
                                        names,occ=get_occupancies(orb1)
                                        for j in occ:
                                            conf_entropy[n]+=-mult[orb1]*j*np.log(j)
                                        for ind,name in enumerate(names):
                                            if(name in mixing_occ.keys()):
                                                mixing_occ[name]+=occ[ind]*mult[orb1]/intersect_orbit_connected_mult[k]
                                            else:
                                                mixing_occ[name]=occ[ind]*mult[orb1]/intersect_orbit_connected_mult[k]
                            
                            for name in mixing_occ.keys():
                                # note in this case multiplicities of intersecting orbits are the same
                                mixing_entropy[n]+=-intersect_orbit_connected_mult[k]*mixing_occ[name]*np.log(mixing_occ[name])

                        else:
                            # Combination of external and internal intersection produces structures, 
                            # for which entropy depends on particular configuration. If total occupancy of intersecting 
                            # sites > 1 sometimes it is possible, but it will be correlated disorder
                            conf_entropy[n]=np.nan
                            mixing_entropy[n]=np.nan
                            break

                elif(dis_type[i]=='VP'):
                    if(calculated_orbits[i]==0):
                        if(internal_intersection[i]==True and len(intersect_orbit_connected[i])==1):
                            # internal intersection, no external intersection
                            calculated_orbits[i]=1
                            _,occ=get_occupancies(orb)
                            for j in occ:
                                mixing_entropy[n]+=-mult[orb]*j*np.log(j*intersect_index[i][0])
                                conf_entropy[n]+=-mult[orb]*j*np.log(j)    
                            mixing_entropy[n]+=-mult[orb]/intersect_index[i][0]*(1-intersect_index[i][0]*j)*np.log(1-j*intersect_index[i][0])
                            conf_entropy[n]+=-mult[orb]/intersect_index[i][0]*(1-intersect_index[i][0]*j)*np.log(1-j*intersect_index[i][0])
                        elif(len(intersect_orbit_connected[i])>1 and intersect_orbit_connected_occ[i]<=1):
                            # external intersection
                            mixing_occ=0
                            for orb1 in intersect_orbit_connected[i]:
                                k=orbits.index(orb1)
                                if(internal_intersection[k]==False):
                                    # no internal intersection
                                    if(calculated_orbits[k]==0):
                                        calculated_orbits[k]=1
                                        _,occ=get_occupancies(orb1)
                                        for j in occ:
                                            conf_entropy[n]+=-mult[orb1]*j*np.log(j)
                                            mixing_occ+=j
                                else:
                                    # internal intersection
                                    if(calculated_orbits[k]==0):
                                        calculated_orbits[k]=1
                                        _,occ=get_occcupancies(orb1)
                                        for j in occ:
                                            conf_entropy[n]+=-mult[orb1]*j*np.log(j)
                                            mixing_occ+=j*mult[orb1]/intersect_orbit_connected_mult[k]
                                    
                            if(1-mixing_occ>0):
                                conf_entropy[n]+=-mult[orb1]*(1-mixing_occ)*np.log(1-mixing_occ)
                                mixing_entropy[n]+=-mult[orb1]*mixing_occ*np.log(mixing_occ)-mult[orb1]*(1-mixing_occ)*np.log(1-mixing_occ)

                        else:
                            # Combination of external and internal intersection produces structures, 
                            # for which entropy depends on particular configuration
                            mixing_entropy[n]=np.nan
                            conf_entropy[n]=np.nan
                            break

                elif(dis_type[i]=='P'):
                    if(internal_intersection[i]==True and len(intersect_orbit_connected[i])==1):
                        # only internal intersection
                        calculated_orbits[i]=1
                        _,occ=get_occupancies(orb)
                        for j in occ:
                            conf_entropy[n]+=-mult[orb]*j*np.log(j)

                    elif(len(intersect_orbit_connected[i])>1 and intersect_orbit_connected_occ[i]<=1.02):
                        # only external intersection
                        for orb1 in intersect_orbit_connected[i]:
                            k=orbits.index(orb1)
                            if(internal_intersection[k]==False):
                                if(calculated_orbits[k]==0):
                                    calculated_orbits[k]=1
                                    _,occ=get_occupancies(orb1)
                                    for j in occ:
                                        conf_entropy[n]+=-intersect_orbit_connected_occ[k]*j*np.log(j)
                            else:
                                if(calculated_orbits[k]==0):
                                    calculated_orbits[k]=1
                                    _,occ=get_occupancies(orb1)
                                    for j in occ:
                                        conf_entropy[n]+=-mult[orb1]*j*np.log(j)
                                       
                    else:
                        # Combination of external and internal intersection produces structures, 
                        # for which entropy depends on particular configuration
                        mixing_entropy[n]=np.nan
                        conf_entropy[n]=np.nan
                        break

                elif(dis_type[i]=='SVP'):
                    if(calculated_orbits[i]==0):
                        if(internal_intersection[i]==True and len(intersect_orbit_connected[i])==1):
                            # only internal intersection.
                            calculated_orbits[i]=1
                            _,occ=get_occupancies(orb)
                            total_occupancy=0
                            for j in occ:
                                mixing_entropy[n]+=-mult[orb]*j*np.log(j*intersect_index[i][0])
                                conf_entropy[n]+=-mult[orb]*j*np.log(j)
                                total_occupancy+=j
                            if(intersect_index[i][0]*total_occupancy<1):
                                mixing_entropy[n]+=-mult[orb]/intersect_index[i][0]*(1-intersect_index[i][0]*total_occupancy)*np.log(1-total_occupancy*intersect_index[i][0])
                                conf_entropy[n]+=-mult[orb]/intersect_index[i][0]*(1-intersect_index[i][0]*total_occupancy)*np.log(1-total_occupancy*intersect_index[i][0])
                        elif(len(intersect_orbit_connected[i])>1 and intersect_orbit_connected_occ[i]<=1.02):
                            # external intersection. In this case we have different orbits with 
                            # haveing substitution, vacancies and/or positional disorder
                            mixing_occ={}
                            for orb1 in intersect_orbit_connected[i]:
                                k=orbits.index(orb1)
                                if(internal_intersection[k]==False):
                                    if(calculated_orbits[k]==0):
                                        calculated_orbits[k]=1
                                        names,occ=get_occupancies(orb1)
                                        for j in occ:
                                            conf_entropy[n]+=-mult[orb1]*j*np.log(j)
                                        for ind,name in enumerate(names):
                                            if(name in mixing_occ.keys()):
                                                mixing_occ[name]+=occ[ind]
                                            else:
                                                mixing_occ[name]=occ[ind] 
                                else:
                                    if(calculated_orbits[k]==0):
                                        calculated_orbits[k]=1
                                        names,occ=get_occupancies(orb1)
                                        for j in occ:
                                            conf_entropy[n]+=-mult[orb1]*j*np.log(j)
                                        for ind,name in enumerate(names):
                                            if(name in mixing_occ.keys()):
                                                mixing_occ[name]+=occ[ind]*mult[orb1]/intersect_orbit_connected_occ[k]
                                            else:
                                                mixing_occ[name]=occ[ind]*mult[orb1]/intersect_orbit_connected_occ[k]
                                          
                            for name in mixing_occ.keys():
                                # note in this case multiplicities of intersecting orbits are the same
                                mixing_entropy[n]+=-intersect_orbit_connected_occ[i]*mixing_occ[name]*np.log(mixing_occ[name])
                            if(intersect_orbit_connected_occ[i]<1):
                                conf_entropy[n]+=-intersect_orbit_connected_mult[i]*(1-intersect_orbit_connected_occ[i])*np.log(1-intersect_orbit_connected_occ[i])
                                mixing_entropy[n]+=-intersect_orbit_connected_mult[i]*(1-intersect_orbit_connected_occ[i])*np.log(1-intersect_orbit_connected_occ[i])        
                        else:
                            # Combination of external and internal intersection produces structures, 
                            # for which entropy depends on particular configuration
                            mixing_entropy[n]=np.nan
                            conf_entropy[n]=np.nan
                            break
              
            comp=Composition(df.iloc[n]['Formula'])
            natoms=np.sum(list(comp.as_dict().values()))
            x=mixing_entropy[n]/z/natoms
            y=conf_entropy[n]/z/natoms
            mixing_entropy[n]=x
            conf_entropy[n]=y
        
        except:
            mixing_entropy[n]=np.nan
            conf_entropy[n]=np.nan
        
    da=df.copy()
    da['mixing_entropy']=mixing_entropy
    da['conf_entropy']=conf_entropy
    return da


def frac_sites_calculation(df):
    '''
    Function to calculate the total number of sites and the fraction of:
    (1) disordered sites (sites='disordered')
    (2) vacancies (sites='vacancies')
    '''
    total_sites=np.zeros(len(df))
    disordered_sites=np.zeros(len(df))
    vacancies=np.zeros(len(df))
    fraction_dis_sites=np.zeros(len(df))
    fraction_vacancies=np.zeros(len(df))
    for n in range(len(df)):
        try:
            mult=df.iloc[n]['multiplicity']
            orbits=df.iloc[n]['orbits']
            dis_type=df.iloc[n]['disorder_type']
            intersect_orbit=df.iloc[n]['intersect_orbit']
            intersect_orbit_connected=df.iloc[n]['intersect_orbit_connected']
            internal_intersection=df.iloc[n]['internal_intersection']
            intersect_index=df.iloc[n]['intersect_index']
            intersect_orbit_connected_mult=df.iloc[n]['intersect_orbit_connected_mult']
            intersect_orbit_connected_occ=df.iloc[n]['intersect_orbit_connected_occ']

            calculated_orbits=np.zeros(len(orbits))
            
            for i,orb in enumerate(orbits):
                if(calculated_orbits[i]==0):
                    if(dis_type[i]=='O'):
                        calculated_orbits[i]=1
                        total_sites[n]+=mult[orb]
                    elif(dis_type[i]=='S'):
                        calculated_orbits[i]=1
                        total_sites[n]+=mult[orb]
                        disordered_sites[n]+=mult[orb]
                    elif(dis_type[i]=='V'):
                        calculated_orbits[i]=1
                        total_sites[n]+=mult[orb]
                        disordered_sites[n]+=mult[orb]
                        vacancies[n]+=mult[orb]
                    elif(dis_type[i]=='SV'):
                        calculated_orbits[i]=1
                        total_sites[n]+=mult[orb]
                        disordered_sites[n]+=mult[orb]
                        vacancies[n]+=mult[orb]
                    # and now we need to deal with positional disorder
                    elif(dis_type[i]=='P' or dis_type[i]=='SP'):
                        total_sites[n]+=intersect_orbit_connected_mult[i]
                        disordered_sites[n]+=intersect_orbit_connected_mult[i]
                        for orb1 in intersect_orbit_connected[i]:
                            k=orbits.index(orb1)
                            calculated_orbits[k]=1
                    else:
                        total_sites[n]+=intersect_orbit_connected_mult[i]
                        disordered_sites[n]+=intersect_orbit_connected_mult[i]
                        vacancies[n]+=intersect_orbit_connected_mult[i]
                        for orb1 in intersect_orbit_connected[i]:
                            k=orbits.index(orb1)
                            calculated_orbits[k]=1
                            
            if(total_sites[n]>0):
                fraction_dis_sites[n]=disordered_sites[n]/total_sites[n]
                fraction_vacancies[n]=vacancies[n]/total_sites[n]
            else:
                fraction_vacancies[n]=np.nan
                fraction_dis_sites[n]=np.nan
            
        except:
            total_sites[n]=np.nan
            fraction_vacancies[n]=np.nan
            fraction_dis_sites[n]=np.nan
    da=df.copy()
    da['total_sites']=total_sites
    da['fraction_dis_sites']=fraction_dis_sites
    da['fraction_vacancies']=fraction_vacancies
        
    return da
            

In [63]:
df=entropy_calulation(df)

/var/folders/41/wzm8qn9d6dn8yhp48bnwvj380000gn/T/ipykernel_31722/2251546788.py:134: RuntimeWarning: invalid value encountered in log
  mixing_entropy[n]+=-mult[orb]/intersect_index[i][0]*(1-intersect_index[i][0]*j)*np.log(1-j*intersect_index[i][0])
/var/folders/41/wzm8qn9d6dn8yhp48bnwvj380000gn/T/ipykernel_31722/2251546788.py:135: RuntimeWarning: invalid value encountered in log
  conf_entropy[n]+=-mult[orb]/intersect_index[i][0]*(1-intersect_index[i][0]*j)*np.log(1-j*intersect_index[i][0])
/var/folders/41/wzm8qn9d6dn8yhp48bnwvj380000gn/T/ipykernel_31722/2251546788.py:53: RuntimeWarning: divide by zero encountered in log
  mixing_entropy[n]+=-mult[orb]*j*np.log(j)
/var/folders/41/wzm8qn9d6dn8yhp48bnwvj380000gn/T/ipykernel_31722/2251546788.py:53: RuntimeWarning: invalid value encountered in scalar multiply
  mixing_entropy[n]+=-mult[orb]*j*np.log(j)
/var/folders/41/wzm8qn9d6dn8yhp48bnwvj380000gn/T/ipykernel_31722/2251546788.py:54: RuntimeWarning: divide by zero encountered in log
  conf

In [64]:
df=frac_sites_calculation(df)

In [65]:
df.tail(50)

,ID,Formula,disorder_type,disorder_set,symmetry,group_num,Z,orbits,multiplicity,intersect_orbit,...,magic_number,intersect_orbit_connected,intersect_index,intersect_orbit_connected_mult,intersect_orbit_connected_occ,mixing_entropy,conf_entropy,total_sites,fraction_dis_sites,fraction_vacancies
31674,53493,Fe3 Li1 S8 Sn2,"[S, S, O]","{S, O}",F d -3 m S,227,4,"[Li+:0.500, Fe3+:0.500, Fe2+:0.500, Sn4+:0.500...","{'Li+:0.500, Fe3+:0.500': 8, 'Fe2+:0.500, Sn4+...","[[], [], []]",...,"[8.0, 16.0, 32.0]","[{Li+:0.500, Fe3+:0.500}, {Fe2+:0.500, Sn4+:0....","[[1], [1], [1]]","[8, 16, 32]","[1.0, 1.0, 1.0]",0.297063,0.297063,56.0,0.428571,0.000000
31675,157187,H1 Al3 Ca1.89 O13 Si3 Sr0.11,"[O, O, O, S, S, O]","{S, O}",P n m a,62,4,"[H+, Al3+, Si4+, Sr2+:0.090, Ca2+:0.910, Sr2+:...","{'H+': 4, 'Al3+': 12, 'Si4+': 12, 'Sr2+:0.090,...","[[], [], [], [], [], []]",...,"[4.0, 12.0, 12.0, 4.0, 4.0, 52.0]","[{H+}, {Al3+}, {Si4+}, {Sr2+:0.090, Ca2+:0.910...","[[1], [1], [1], [1], [1], [1]]","[4, 12, 12, 4, 4, 52]","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0]",0.018208,0.018208,88.0,0.090909,0.000000
31676,203208,C12 H24 Cd3 Ir2 N12 O12,"[COM, V, V, COM, SV, V]","{V, SV, COM}",F m -3 m,225,1,"[O2-:0.250, C2+:0.500, Cd2+:0.750, N3-:0.500, ...","{'O2-:0.250': 32, 'C2+:0.500': 24, 'Cd2+:0.750...","[[N3-:0.500], [], [], [O2-:0.250], [], []]",...,"[8.0, 12.0, 3.0, 12.0, 3.0, 3.007999999999999]","[{O2-:0.250, N3-:0.500}, {C2+:0.500}, {Cd2+:0....","[[1], [1], [1], [1], [1], [1]]","[32, 24, 4, 32, 4, 32]","[nan, 0.5, 0.75, nan, 0.75, 0.09]",NaN,NaN,96.0,1.000000,1.000000
31677,420852,Ag2 I6 O18 Ti1,"[O, O, O, O]",{O},R -3 H,148,3,"[Ag+, Ti4+, I5+, O2-]","{'Ag+': 6, 'Ti4+': 3, 'I5+': 18, 'O2-': 54}","[[], [], [], []]",...,"[6.0, 3.0, 18.0, 54.0]","[{Ag+}, {Ti4+}, {I5+}, {O2-}]","[[1], [1], [1], [1]]","[6, 3, 18, 54]","[1.0, 1.0, 1.0, 1.0]",0.000000,0.000000,81.0,0.000000,0.000000
31678,647865,Os0.6 W1.4,"[O, S]","{S, O}",P 42/m n m,136,15,"[W0+, Os0+:0.900, W0+:0.100]","{'W0+': 20, 'Os0+:0.900, W0+:0.100': 10}","[[], []]",...,"[20.0, 10.0]","[{W0+}, {Os0+:0.900, W0+:0.100}]","[[1], [1]]","[20, 10]","[1.0, 1.0]",0.108361,0.108361,30.0,0.333333,0.000000
31679,659483,B6 Ba1,"[O, O]",{O},P m -3 m,221,1,"[Ba0+, B0+]","{'Ba0+': 1, 'B0+': 6}","[[], []]",...,"[1.0, 6.0]","[{Ba0+}, {B0+}]","[[1], [1]]","[1, 6]","[1.0, 1.0]",0.000000,0.000000,7.0,0.000000,0.000000
31680,411457,Ho1 In1 Pt1,"[O, O, O]",{O},P -6 2 m,189,3,"[Ho0+, In0+, Pt0+]","{'Ho0+': 3, 'In0+': 3, 'Pt0+': 3}","[[], [], []]",...,"[3.0, 3.0, 3.0]","[{Ho0+}, {In0+}, {Pt0+}]","[[1], [1], [1]]","[3, 3, 3]","[1.0, 1.0, 1.0]",0.000000,0.000000,9.0,0.000000,0.000000
31681,607299,Al3 Dy0.98 Gd0.02,"[S, O]","{S, O}",P 63/m m c,194,4,"[Gd0+:0.020, Dy0+:0.980, Al0+]","{'Gd0+:0.020, Dy0+:0.980': 4, 'Al0+': 12}","[[], []]",...,"[4.0, 12.0]","[{Gd0+:0.020, Dy0+:0.980}, {Al0+}]","[[1], [1]]","[4, 12]","[1.0, 1.0]",0.024510,0.024510,16.0,0.250000,0.000000
31682,35716,F6 Fe1 Zr1,"[O, O, O]",{O},F m -3 m,225,4,"[Zr4+, F-, Fe2+]","{'Zr4+': 4, 'F-': 24, 'Fe2+': 4}","[[], [], []]",...,"[4.0, 24.0, 4.0]","[{Zr4+}, {F-}, {Fe2+}]","[[1], [1], [1]]","[4, 24, 4]","[1.0, 1.0, 1.0]",0.000000,0.000000,32.0,0.000000,0.000000
31683,252721,C2 Ca1,"[O, O]",{O},C 1 2/c 1,15,4,"[C0+, Ca0+]","{'C0+': 8, 'Ca0+': 4}","[[], []]",...,"[8.0, 4.0]","[{C0+}, {Ca0+}]","[[1], [1]]","[8, 4]","[1.0, 1.0]",0.000000,0.000000,12.0,0.000000,0.000000


In [66]:
df.to_csv('disorder_data_180000_end.csv')